# Clustering Crypto

In [77]:
# Initial imports
import pandas as pd
import numpy as np
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas


### Deliverable 1: Preprocessing the Data for PCA

In [78]:
# Load the crypto_data.csv dataset.
file_path = "Resources/crypto_data.csv"
crypto_df = pd.read_csv(file_path, index_col=[0])
crypto_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [79]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df["IsTrading"]==True]
crypto_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [80]:
crypto_df.shape

(1144, 6)

In [81]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df = crypto_df.replace(r'^s*$', float('NaN'), regex = True)
crypto_df = crypto_df[crypto_df['Algorithm'].notna()]
crypto_df.shape

(1144, 6)

In [82]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(['IsTrading'],1)
crypto_df.head()

/var/folders/f_/5twpc38j10g0xh1n_x3nbrrh0000gn/T/ipykernel_30894/4036950287.py:2: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [83]:
crypto_df.shape

(1144, 5)

In [84]:
# Remove rows that have at least 1 null value.
crypto_df.dropna(inplace = True)
crypto_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000


In [85]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df['TotalCoinsMined'] > 0]
crypto_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000
ZEC,ZCash,Equihash,PoW,7.383056e+06,21000000


In [86]:
crypto_df.shape

(532, 5)

In [87]:
# Create a new DataFrame that holds only the cryptocurrencies names.
names_df = pd.DataFrame(
    index= crypto_df.index, 
    data = crypto_df['CoinName']
)
names_df.head(10)

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
LTC,Litecoin
DASH,Dash
XMR,Monero
ETC,Ethereum Classic
ZEC,ZCash


In [88]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop(['CoinName'],1)
crypto_df.head(10)

/var/folders/f_/5twpc38j10g0xh1n_x3nbrrh0000gn/T/ipykernel_30894/1112850185.py:2: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [89]:
crypto_df.shape

(532, 4)

In [90]:
# Use get_dummies() to create variables for text features.
x = pd.get_dummies(crypto_df,columns=['Algorithm','ProofType'])
x.shape
crypto_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [91]:
crypto_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [92]:
# Standardize the data with StandardScaler().
x = StandardScaler().fit_transform(x)
with np.printoptions(threshold=np.inf):
        print(x)

[[-1.17108170e-01 -1.52870298e-01 -4.33963037e-02 -4.33963037e-02
  -4.33963037e-02 -6.14295117e-02 -7.53065569e-02 -4.33963037e-02
  -6.14295117e-02 -6.14295117e-02 -4.33963037e-02 -4.33963037e-02
  -1.92450090e-01 -6.14295117e-02 -9.74046510e-02 -4.33963037e-02
  -1.15470054e-01 -7.53065569e-02 -4.33963037e-02 -4.33963037e-02
  -1.51910905e-01 -4.33963037e-02 -1.31180841e-01 -4.33963037e-02
  -4.33963037e-02 -8.70388280e-02 -4.33963037e-02 -4.33963037e-02
  -4.33963037e-02 -4.33963037e-02 -6.14295117e-02 -4.33963037e-02
  -8.70388280e-02 -8.70388280e-02 -8.70388280e-02 -4.33963037e-02
  -1.31180841e-01 -1.38409133e-01 -1.38409133e-01 -4.33963037e-02
  -6.14295117e-02 -4.33963037e-02 -7.53065569e-02 -1.81685742e-01
  -4.33963037e-02 -4.33963037e-02 -4.33963037e-02 -7.53065569e-02
  -1.58266135e-01 -3.14918329e-01 -4.33963037e-02 -8.70388280e-02
  -7.53065569e-02 -6.14295117e-02  1.38675049e+00 -4.33963037e-02
  -4.33963037e-02 -6.14295117e-02 -4.33963037e-02 -4.33963037e-02
  -4.33963

### Deliverable 2: Reducing Data Dimensions Using PCA

In [93]:
# Using PCA to reduce dimension to three principal components.
# Initialize PCA model
pca = PCA(n_components=3)
# Get three principal components for the crypto data.
crypto_pca = pca.fit_transform(x)

In [94]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(
    index=crypto_df.index,
    data = crypto_pca, columns=["PC1","PC2", "PC3"]
)
pcs_df.head()

,PC1,PC2,PC3
42,-0.337262,1.029240,-0.583568
404,-0.320654,1.029653,-0.584013
1337,2.283379,1.740148,-0.666664
BTC,-0.148291,-1.328035,0.171246
ETH,-0.146377,-2.024083,0.404050


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [95]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)



In [96]:
# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [97]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)


# Predict clusters
predictions = model.predict(pcs_df)
predictions

array([3, 3, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 3, 3, 0, 3, 0, 0, 3, 3, 0, 0,
       0, 0, 0, 3, 0, 0, 0, 3, 0, 3, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 3, 3,
       0, 0, 0, 0, 0, 3, 3, 0, 3, 0, 0, 0, 0, 3, 0, 0, 3, 0, 3, 3, 3, 0,
       0, 0, 3, 3, 3, 3, 3, 0, 0, 0, 3, 3, 0, 3, 0, 3, 3, 0, 0, 0, 0, 3,
       3, 0, 3, 0, 0, 3, 3, 0, 3, 3, 0, 0, 3, 3, 0, 3, 3, 0, 3, 0, 3, 0,
       3, 0, 3, 3, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 3,
       0, 3, 0, 0, 3, 0, 3, 0, 3, 3, 0, 0, 3, 0, 0, 3, 3, 0, 3, 0, 3, 3,
       3, 0, 0, 0, 0, 3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3,
       3, 0, 3, 0, 3, 3, 0, 3, 0, 3, 3, 0, 3, 0, 3, 0, 3, 0, 3, 3, 3, 3,
       0, 3, 3, 3, 3, 3, 0, 0, 3, 3, 0, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3,
       3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 0, 0, 3, 3, 3, 3, 0, 3, 0, 3,
       3, 0, 3, 0, 0, 3, 0, 0, 3, 0, 3, 3, 3, 0, 3, 3, 0, 3, 3, 3, 3, 3,
       3, 3, 0, 3, 0, 3, 3, 3, 3, 0, 3, 0, 3, 0, 0, 0, 0, 3, 0, 3, 3, 0,
       3, 0, 0, 0, 3, 0, 3, 0, 0, 0, 3, 0, 3, 0, 3,

In [98]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df,pcs_df], axis = 1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df = pd.concat([clustered_df,names_df], axis = 1)

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['Class']=predictions

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.337262,1.029240,-0.583568,42 Coin,3
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.320654,1.029653,-0.584013,404Coin,3
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.283379,1.740148,-0.666664,EliteCoin,3
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.148291,-1.328035,0.171246,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.146377,-2.024083,0.404050,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.164602,-1.126483,-0.001782,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.399767,1.237469,-0.519107,Dash,3
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.151695,-2.217796,0.457443,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.144825,-2.024153,0.404019,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.174096,-1.943160,0.389728,ZCash,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [99]:
# Creating a 3D-Scatter with the PCA data and the clusters
# Plotting the 2D-Scatter with x="Annual Income" and y="Spending Score
fig = px.scatter_3d(
    clustered_df,
    x=clustered_df["PC1"],
    y=clustered_df["PC2"],
    z=clustered_df["PC3"],
    hover_name="CoinName",
    hover_data=["Algorithm"],
    color="Class",
    symbol="Class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [100]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class' ], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [101]:
# Print the total number of tradable cryptocurrencies.
len(clustered_df)

532

In [102]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
TotalCoinSupply_df = pd.DataFrame(
    index= clustered_df.index, 
    data = clustered_df['TotalCoinSupply'])

TotalCoinSupply_scaled =  MinMaxScaler(feature_range=(0, 1)).fit_transform(TotalCoinSupply_df)
TotalCoinSupply_scaled_df = pd.DataFrame(index= clustered_df.index,
                                         data = TotalCoinSupply_scaled, columns=['TotalCoinSupply'])

TotalCoinsMined_df = pd.DataFrame(
    index= clustered_df.index, 
    data = clustered_df['TotalCoinsMined'])

TotalCoinsMined_scaled =  MinMaxScaler(feature_range=(0, 1)).fit_transform(TotalCoinsMined_df)
TotalCoinsMined_scaled_df = pd.DataFrame(index= clustered_df.index,
                                         data = TotalCoinsMined_scaled, columns=['TotalCoinsMined'])

In [103]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(
    index=clustered_df.index)
plot_df = pd.concat([TotalCoinSupply_scaled_df,TotalCoinsMined_scaled_df], axis = 1)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df = plot_df.join(clustered_df["CoinName"])

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df = plot_df.join(clustered_df["Class"])

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,3
404,5.320000e-04,0.001066,404Coin,3
1337,3.141593e-01,0.029576,EliteCoin,3
BTC,2.100000e-05,0.000018,Bitcoin,0
ETH,0.000000e+00,0.000109,Ethereum,0
LTC,8.400000e-05,0.000064,Litecoin,0
DASH,2.200000e-05,0.000009,Dash,3
XMR,0.000000e+00,0.000017,Monero,0
ETC,2.100000e-04,0.000115,Ethereum Classic,0
ZEC,2.100000e-05,0.000007,ZCash,0


In [104]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", by="Class")


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply)